In [ ]:
pip install -r requirements.txt

In [2]:
import config_240_records as config_240_records


In [4]:
import importlib
importlib.reload(config_240_records)

<module 'config_240_records' from 'd:\\OVGU _Saurabh\\SEM 3\\Review Project\\Code\\Prompting\\config_240_records.py'>

In [5]:
import numpy as np
import pandas as pd
from langchain_ollama import OllamaLLM
from langchain_core.prompts import PromptTemplate
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
# from config import Config
# from evaluate import evaluate_model

In [6]:
from config_240_records import Config

In [6]:
#from evaluate import evaluate_model

In [7]:
# Load your CSV
df_new = pd.read_csv(Config.file_path)
reviews = df_new["text"].tolist()

In [8]:
prompt = PromptTemplate(
    input_variables=["text"], # the placeholder to be replaced based on prompt template
    template=Config.zero_shot_prompt_template
)

# Load small model
llm = OllamaLLM(model=Config.model)

# Create chain using pipe operator (modern LangChain syntax)
chain = prompt | llm

In [9]:
# Classify each review
results = []
for r in reviews:
    result = chain.invoke({"text": r})
    # Clean the output - extract only "truthful" or "deceptive"
    label = result.strip().lower()
    # Extract first word if model adds extra text
    if " " in label:
        label = label.split()[0]
    # Remove any punctuation
    label = label.strip('.,!?;:')
    print(f"Raw output: {result} -> Cleaned: {label}")
    results.append(label)

In [9]:
df_new.head(10)


,label,hotel,source,text
0,truthful,james,Web,"I have stayed at the James a few times, it is ..."
1,truthful,hilton,Web,The hotel is very impressive upon entering and...
2,truthful,omni,TripAdvisor,My wife and I travelled to Chicago and really ...
3,truthful,homewood,Web,After reading good reviews about I booked a 5 ...
4,truthful,fairmont,TripAdvisor,We recently completed our second stay at the F...
5,truthful,hardrock,Web,The Good LOCATION Views Decor Room Service Fri...
6,truthful,allegro,TripAdvisor,This is best hotel bargin in Chicago if you ar...
7,truthful,homewood,Web,We planned to stay for 5 nights on a family tr...
8,truthful,palmer,Web,After Leaving some important documents in the ...
9,truthful,hyatt,Web,Thank god I got this hotel through priceline. ...


In [10]:
# def clean_label(text):
#     return text.strip().lower().split()[0].strip('.,!?;:')
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

def evaluate_model(df, results):
    # Get true labels from the dataframe
    #y_true = df["deceptive"].tolist()
    y_true = df["label"].tolist()
    y_pred = results

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted', pos_label=None)
    conf_matrix = confusion_matrix(y_true, y_pred)

    return accuracy, f1, conf_matrix

In [17]:
from config_240_records import Config

In [11]:
for label in results:
    print(label)

truthful
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
truthful
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
truthful
deceptive
deceptive
truthful
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
truthful
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
truthful
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
truthful
deceptive
truthful
deceptive
truthful
deceptive
deceptive
deceptive
deceptive
truthful
deceptive
deceptive
deceptive
truthful
deceptive
truthful
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
truthful
truthful
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
deceptive
truthful
deceptive
deceptive
truthful
truthf

In [12]:
accuracy, f1, conf_matrix = evaluate_model(df_new, results)

print("=" * 50)
print("ZERO-SHOT PROMPTING RESULTS")
print("=" * 50)
print("Accuracy:", accuracy)
print("\nF1 Score:", f1)
print("\nConfusion Matrix:")
print(conf_matrix)
print("=" * 50)

ZERO-SHOT PROMPTING RESULTS
Accuracy: 0.4625

F1 Score: 0.38118366612700133

Confusion Matrix:
[[ 99  21]
 [108  12]]


## One-Shot Prompting
Using one example to guide the model's classification

In [13]:
one_shot_prompt = PromptTemplate(
    input_variables=["text"],
    template = Config.one_shot_prompt_template
)
# Create one-shot chain
one_shot_chain = one_shot_prompt | llm

In [14]:
# Classify each review
one_shot_results = []
for r in reviews:
    result = one_shot_chain.invoke({"text": r})
    # Clean the output - extract only "truthful" or "deceptive"
    label = result.strip().lower()
    # Extract first word if model adds extra text
    if " " in label:
        label = label.split()[0]
    # Remove any punctuation
    label = label.strip('.,!?;:')
    print(f"Raw output: {result} -> Cleaned: {label}")
    one_shot_results.append(label)

Raw output: Truthful. -> Cleaned: truthful
Raw output: Truthful. -> Cleaned: truthful
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive.

The language used here is overly positive and lacks specific details about what went wrong at the hotel. The reviewer mentions that everything was fine for thirty years ago, implying a significant improvement from past experiences, but does not provide any concrete examples or complaints to support this claim. Additionally, the tone of the review seems more focused on expressing enthusiasm than providing criticism or negative feedback, which is another characteristic of deceptive review

In [15]:
df_new.head(10)

,label,hotel,source,text
0,truthful,james,Web,"I have stayed at the James a few times, it is ..."
1,truthful,hilton,Web,The hotel is very impressive upon entering and...
2,truthful,omni,TripAdvisor,My wife and I travelled to Chicago and really ...
3,truthful,homewood,Web,After reading good reviews about I booked a 5 ...
4,truthful,fairmont,TripAdvisor,We recently completed our second stay at the F...
5,truthful,hardrock,Web,The Good LOCATION Views Decor Room Service Fri...
6,truthful,allegro,TripAdvisor,This is best hotel bargin in Chicago if you ar...
7,truthful,homewood,Web,We planned to stay for 5 nights on a family tr...
8,truthful,palmer,Web,After Leaving some important documents in the ...
9,truthful,hyatt,Web,Thank god I got this hotel through priceline. ...


In [16]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

def evaluate_model(df, results):
    # Get true labels from the dataframe
    #y_true = df["deceptive"].tolist()
    y_true = df["label"].tolist()
    y_pred = results

    # Calculate metrics
    accuracy = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted', pos_label=None)
    conf_matrix = confusion_matrix(y_true, y_pred)

    return accuracy, f1, conf_matrix

In [17]:
accuracy_one_shot, f1_one_shot, conf_matrix_one_shot = evaluate_model(df_new, one_shot_results)

print("=" * 50)
print("ONE-SHOT PROMPTING RESULTS")
print("=" * 50)
print("Accuracy:", accuracy_one_shot)
print("\nF1 Score:", f1_one_shot)
print("\nConfusion Matrix:")
print(conf_matrix_one_shot)
print("=" * 50)

ONE-SHOT PROMPTING RESULTS
Accuracy: 0.5041666666666667

F1 Score: 0.4073582204146002

Confusion Matrix:
[[109  11]
 [108  12]]


## Few-Shot Prompting
Using multiple examples to guide the model's classification

In [175]:
few_shot_prompt = PromptTemplate(
    input_variables=["text"],
    template=Config.few_shot_prompt_template
)

# Create few-shot chain
few_shot_chain = few_shot_prompt | llm

In [24]:
# Classify reviews using few-shot prompting
few_shot_results = []
for r in reviews:
    result = few_shot_chain.invoke({"text": r})
    # Clean the output
    label = result.strip().lower()
    if " " in label:
        label = label.split()[0]
    label = label.strip('.,!?;:')
    print(f"Raw output: {result} -> Cleaned: {label}")
    few_shot_results.append(label)

Raw output: Truthful. -> Cleaned: truthful
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Truthful -> Cleaned: truthful
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Truthful. -> Cleaned: truthful
Raw output: Truthful -> Cleaned: truthful
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Truthful. -> Cleaned: truthful
Raw output: Truthful -> Cleaned: truthful
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive. -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: deceptive
Raw output: Deceptive -> Cleaned: dece

In [25]:
accuracy_few_shot, f1_few_shot, conf_matrix_few_shot = evaluate_model(df_new, few_shot_results)

print("=" * 50)
print("FEW-SHOT PROMPTING RESULTS")
print("=" * 50)
print("Accuracy:", accuracy_few_shot)
print("\nF1 Score:", f1_few_shot)
print("\nConfusion Matrix:")
print(conf_matrix_few_shot)
print("=" * 50)

FEW-SHOT PROMPTING RESULTS
Accuracy: 0.5166666666666667

F1 Score: 0.45625

Confusion Matrix:
[[102  18]
 [ 98  22]]


## Compare All Approaches
Compare zero-shot, one-shot, and few-shot results

In [26]:
# Compare all approaches
comparison_df = pd.DataFrame({
    'Approach': ['Zero-Shot', 'One-Shot', 'Few-Shot'],
    'Accuracy': [accuracy, accuracy_one_shot, accuracy_few_shot],
    'F1 Score': [f1, f1_one_shot, f1_few_shot]
})

print("=" * 60)
print("COMPARISON OF ALL PROMPTING APPROACHES")
print("=" * 60)
print(comparison_df.to_string(index=False))
print("=" * 60)

# Find best approach
best_idx = comparison_df['Accuracy'].idxmax()
best_approach = comparison_df.loc[best_idx, 'Approach']
best_accuracy = comparison_df.loc[best_idx, 'Accuracy']
print(f"\nBest Approach: {best_approach} (Accuracy: {best_accuracy:.4f})")

COMPARISON OF ALL PROMPTING APPROACHES
 Approach  Accuracy  F1 Score
Zero-Shot  0.462500  0.381184
 One-Shot  0.504167  0.407358
 Few-Shot  0.516667  0.456250

Best Approach: Few-Shot (Accuracy: 0.5167)
